# Actividad en Clase Nro.5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Extracción de los Datos

In [ ]:
semana1 = pd.read_csv('./semana1.txt', sep='\t', encoding='utf-16', parse_dates=['created_time'])

semana2 = pd.read_excel('./semana2.xlsx', parse_dates=['created_time'])

## Limpieza, Transformación y Estructuración de los Datos 

In [ ]:
semana1.columns = semana1.columns.str.lower()
semana2.columns = semana2.columns.str.lower()

In [ ]:
common_columns = ['id', 'created_time', 'platform', 'nivel_académico', 
                   '¿en_qué_franja_horaria_podemos_contactarte?', 
                   'medio_de_contacto', 'nombre', 'apellido', 
                   'número_de_teléfono', 'ciudad', 'correo_electrónico', 
                   'lead_status']

semana1 = semana1[common_columns]
semana2 = semana2[common_columns]

In [ ]:
semana1['platform'] = semana1['platform'].str.replace('ag:', '').str.strip()
semana2['platform'] = semana2['platform'].str.strip()

semana1['ciudad'] = semana1['ciudad'].str.strip().str.title()
semana2['ciudad'] = semana2['ciudad'].str.strip().str.title()

In [ ]:
semana1['franja_horaria'] = semana1['¿en_qué_franja_horaria_podemos_contactarte?'].str.replace('_', '').str.strip()
semana2['franja_horaria'] = semana2['¿en_qué_franja_horaria_podemos_contactarte?'].str.replace('_', '').str.strip()

In [ ]:
df = pd.concat([semana1, semana2], ignore_index=True)

In [ ]:
df['created_time'] = pd.to_datetime(df['created_time'], errors='coerce')
df['hora'] = df['created_time'].dt.hour
df['dia_semana'] = df['created_time'].dt.day_name()
df['fecha'] = df['created_time'].dt.date

In [ ]:
df["nombre"].head(20)

In [ ]:
df['nombre_completo'] = df['nombre'] + ' ' + df['apellido']
df['genero'] = df['nombre'].apply(lambda x: 'F' if x.split()[0].lower() in ['marjorie', 'katherine', 'paola', 'gabriela', 'ligia', 'nathalie', 'silvia', 'laura'] else 'M')

## Visualización de los Datos


### Accesos por días de la semana

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='dia_semana', order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Accesos por día de la semana')
plt.xlabel('Día de la semana')
plt.ylabel('Número de accesos')
plt.show()

### Accesos por hora del día

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='hora', bins=24, kde=True)
plt.title('Distribución de accesos por hora del día')
plt.xlabel('Hora del día')
plt.ylabel('Número de accesos')
plt.xticks(range(24))
plt.show()

### Franja horaria preferida para contacto

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='franja_horaria', order=['mañana', 'tarde', 'noche'])
plt.title('Preferencia de franja horaria para contacto')
plt.xlabel('Franja horaria')
plt.ylabel('Número de usuarios')
plt.show()

### Acccesos por género

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='genero')
plt.title('Distribución de accesos por género')
plt.xlabel('Género')
plt.ylabel('Número de accesos')
plt.show()

### Distribución horaria por género

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='hora', hue='genero', bins=24, multiple='stack')
plt.title('Distribución horaria por género')
plt.xlabel('Hora del día')
plt.ylabel('Número de accesos')
plt.xticks(range(24))
plt.show()

### Accesos por plataforma

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='platform')
plt.title('Distribución de accesos por plataforma')
plt.xlabel('Plataforma')
plt.ylabel('Número de accesos')
plt.show()

### Ciudades con más accesos

In [ ]:
top_ciudades = df['ciudad'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_ciudades.values, y=top_ciudades.index)
plt.title('Top 10 ciudades con más accesos')
plt.xlabel('Número de accesos')
plt.ylabel('Ciudad')
plt.show()

### Determinar perfiles de usuario mediante clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
df_cluster = df.groupby('correo_electrónico').agg({
    'hora': ['mean', 'std'],
    'created_time': 'count'
}).fillna(0)
df_cluster.columns = ['hora_promedio', 'hora_std', 'frecuencia']

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df_cluster)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df_cluster['cluster'] = kmeans.fit_predict(X)

In [ ]:
df_cluster.head(15)

### Actividad por días y horas

In [ ]:
heatmap_data = df.groupby(['dia_semana', 'hora']).size().unstack().fillna(0)

# Ordenar días
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex(day_order).fillna(0)

plt.figure(figsize=(15, 8))
sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt='g')
plt.title('Actividad por día y hora')
plt.xlabel('Hora del día')
plt.ylabel('Día de la semana')
plt.show()